# Demonstrating the module system in Python and Nipype

## First, let's demonstrate that bet, a tool from the fsl toolkit, is not on our path

In [1]:
!bet

/bin/bash: bet: command not found


# Now we can use lmod to load fsl in a specific version

In [2]:
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.4')
await lmod.list()

['fsl/6.0.4']

In [3]:
!bet


Usage:    bet <input> <output> [options]

Main bet2 options:
  -o          generate brain surface outline overlaid onto original image
  -m          generate binary brain mask
  -s          generate approximate skull image
  -n          don't generate segmented brain image output
  -f <f>      fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates
  -g <g>      vertical gradient in fractional intensity threshold (-1->1); default=0; positive values give larger brain outline at bottom, smaller at top
  -r <r>      head radius (mm not voxels); initial surface sphere is set to half of this
  -c <x y z>  centre-of-gravity (voxels not mm) of initial mesh surface.
  -t          apply thresholding to segmented brain image and mask
  -e          generates brain surface as mesh in .vtk format

Variations on default bet2 functionality (mutually exclusive options):
  (default)   just run bet2
  -R          robust brain centre estimation (iterates BE

## Load afni as well

In [4]:
await lmod.load('afni/22.3.06')
await lmod.list()

['fsl/6.0.4', 'afni/22.3.06']

## download test data

In [5]:
!wget https://objectstorage.us-ashburn-1.oraclecloud.com/n/idrvm4tkz2a8/b/TOMCAT/o/TOMCAT_DIB/sub-01/ses-01_7T/anat/sub-01_ses-01_7T_T1w_defaced.nii.gz -O /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz 
!gunzip /neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz 

--2022-12-04 01:46:35--  https://objectstorage.us-ashburn-1.oraclecloud.com/n/idrvm4tkz2a8/b/TOMCAT/o/TOMCAT_DIB/sub-01/ses-01_7T/anat/sub-01_ses-01_7T_T1w_defaced.nii.gz
Resolving objectstorage.us-ashburn-1.oraclecloud.com (objectstorage.us-ashburn-1.oraclecloud.com)... 134.70.28.1, 134.70.24.1, 134.70.32.1
Connecting to objectstorage.us-ashburn-1.oraclecloud.com (objectstorage.us-ashburn-1.oraclecloud.com)|134.70.28.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72682635 (69M) [application/octet-stream]
Saving to: ‘/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz’

/neurodesktop-stora 100%[===================>]  69.32M  20.2MB/s    in 3.4s    

2022-12-04 01:46:39 (20.2 MB/s) - ‘/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii.gz’ saved [72682635/72682635]



In [6]:
!ls /home/jovyan/neurodesktop-storage

sub-01_ses-01_7T_T1w_defaced.nii


## run nipype pipeline

In [7]:
from nipype.interfaces import fsl
from nipype.interfaces import afni

btr = fsl.BET()
btr.inputs.in_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii'
btr.inputs.frac = 0.4
btr.inputs.out_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_brain.nii'
res = btr.run() 

edge3 = afni.Edge3()
edge3.inputs.in_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii'
edge3.inputs.out_file = '/home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced_edges.nii'
edge3.inputs.datum = 'byte'
res = edge3.run()

221204-01:46:44,714 nipype.interface WARNING:
	 FSLOUTPUTTYPE environment variable is not set. Setting FSLOUTPUTTYPE=NIFTI
221204-01:46:54,492 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.492130:++ 3dedge3: AFNI version=AFNI_22.3.06 (Nov 15 2022) [64-bit]
221204-01:46:54,493 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.493828:** AFNI converts NIFTI_datatype=4 (INT16) in file /home/jovyan/neurodesktop-storage/sub-01_ses-01_7T_T1w_defaced.nii to FLOAT32
221204-01:46:54,494 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.493828:     Warnings of this type will be muted for this session.
221204-01:46:54,495 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.493828:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
221204-01:46:54,497 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.497256:*+ WARNING:   If you are performing spatial transformations on an oblique dset,
221204-01:46:54,498 nipype.interface INFO:
	 stderr 2022-12-04T01:46:54.497256:  suc

In [8]:
!ls /home/jovyan/neurodesktop-storage

sub-01_ses-01_7T_T1w_defaced.nii
sub-01_ses-01_7T_T1w_defaced_brain.nii.gz
sub-01_ses-01_7T_T1w_defaced_edges.nii
